In [25]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

In [28]:
from inference_auth_token import get_access_token

access_token = get_access_token()
base_url="https://data-portal-dev.cels.anl.gov/resource_server/sophia/vllm/v1"
model ='meta-llama/Meta-Llama-3.1-70B-Instruct'

try:
    llm = ChatOpenAI(model=model, base_url=base_url, api_key=access_token)
    print(f"Successfully loaded model: {model} from {base_url}")

except Exception as e:
    print(f"Error with loading {model}")
    print(e)

Successfully loaded model: meta-llama/Meta-Llama-3.1-70B-Instruct from https://data-portal-dev.cels.anl.gov/resource_server/sophia/vllm/v1


In [ ]:
# Using pre-built ReAct agent from LangGraph example: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
from typing import Literal
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")

# Define the tools and graph

tools = [get_weather]
weather_graph = create_react_agent(llm, tools=tools)

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
            
inputs = {"messages": [("user", "what is the weather in sf")]}
print_stream(weather_graph.stream(inputs, stream_mode="values"))

In [30]:
!pip install pubchempy

  Using cached pubchempy-1.0.4-py3-none-any.whl


In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

@tool
def molecule_name_to_smiles(name: str) -> str:
    """Convert a molecule name SMILES format.

    Args:
        name (str): molecule name.

    Returns:
        str: SMILES string.
    """
    return pubchempy.get_compounds(str(name), "name")[0].canonical_smiles

tools = [molecule_name_to_smiles]
chem_graph = create_react_agent(llm, tools=tools)

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
            
inputs = {"messages": [("user", "what is SMILES string of Carbon Dioxide?")]}
print_stream(chem_graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

what is SMILES string of Carbon Dioxide?
